# S03 — Safety Feature Engineering

Строим фичи на уровнях **point / hex / edge**.  
Опционально вычисляем **микро‑сегменты** (для HBR/HCT) в быстром приближении.

In [ ]:
%run ./S00_setup.ipynb

In [ ]:
# 1) Загрузка исходных и (если есть) map-matched данных
read_kwargs = dict(sep=",", engine="c",
                   dtype={"randomized_id":"int64","lat":"float64","lng":"float64","alt":"float64","spd":"float64","azm":"float64"},
                   header=0)
try:
    sample = pd.read_csv(DATA_PATH, nrows=5, **read_kwargs)
    exp = ["randomized_id","lat","lng","alt","spd","azm"]
    if list(sample.columns[:6]) != exp:
        read_kwargs.update({"header":None,"names":exp})
except Exception:
    read_kwargs.update({"header":None,"names":["randomized_id","lat","lng","alt","spd","azm"]})
df = pd.read_csv(DATA_PATH, **read_kwargs)
print("Base data:", df.shape)

mm = None
if MATCHED_PARQUET.exists():
    mm = pd.read_parquet(MATCHED_PARQUET)
    print("Matched loaded:", mm.shape)

In [ ]:
# 2) Point-level features
pt = (mm if mm is not None else df).copy()
pt["spd_clip"] = clip_series_by_quantiles(pt["spd"].astype(float))
pt["stop_flag"] = pt["spd_clip"] < CONFIG["STOP_SPEED_MS"]
pt["azm"] = pt["azm"] % 360.0
if "edge_bearing" in pt.columns:
    pt["bearing_dev"] = angle_diff_deg(pt["azm"], pt["edge_bearing"])
else:
    pt["bearing_dev"] = np.nan

# alt residual: отклонение от локальной медианы (здесь — глоб. прибл. по кванти и затем z)
alt_med = pt["alt"].median()
pt["alt_residual"] = pt["alt"] - alt_med

# H3 hex id
if h3 is not None:
    res = CONFIG["H3_RES_HEX"]
    pt["h3"] = [h3.geo_to_h3(lat, lng, res) for lat,lng in zip(pt["lat"].values, pt["lng"].values)]
else:
    pt["h3"] = None

# H3 hex id (v3/v4 compatible + safe for Parquet)
try:
    from h3 import h3 as h3lib   # v3
except Exception:
    try:
        import h3 as h3lib       # v4
    except Exception:
        h3lib = None

if h3lib is not None:
    res = CONFIG["H3_RES_HEX"]
    to_cell = getattr(h3lib, "geo_to_h3", None) or getattr(h3lib, "latlng_to_cell", None)
    cells = [to_cell(lat, lng, res) for lat, lng in zip(pt["lat"].values, pt["lng"].values)]
    # Normalize to string for Parquet portability (avoids uint64 issues)
    cell_to_str = getattr(h3lib, "int_to_h3", None) or getattr(h3lib, "cell_to_str", None)
    pt["h3"] = [cell_to_str(c) if cell_to_str else str(c) for c in cells]
else:
    pt["h3"] = None

# Сохраняем point features
pt.to_parquet(POINT_FEATURES_PARQUET, index=False)
print("Saved:", POINT_FEATURES_PARQUET)

In [ ]:
# 3) Micro-segments (FAST_MODE): используем приближенную "пространственную сортировку"
# В отсутствие времени упорядочим точки внутри ID по (lat,lng) сетке и возьмём пары соседей.
FAST_MODE = True
if FAST_MODE:
    pts = pt[["randomized_id","lat","lng","spd","azm"]].copy()
    # Пространственный ключ: округление lat/lng до 5e-4 (~55 м), затем сортировки
    pts["lat_b"] = (pts["lat"] / 5e-4).round().astype(int)
    pts["lng_b"] = (pts["lng"] / 5e-4).round().astype(int)
    pts = pts.sort_values(["randomized_id","lat_b","lng_b"]).reset_index(drop=True)

    # Сдвиги на ±1 в этом порядке как кандидаты пар
    for shift in [1]:
        pts[f"lat_next_{shift}"] = pts["lat"].shift(-shift)
        pts[f"lng_next_{shift}"] = pts["lng"].shift(-shift)
        pts[f"spd_next_{shift}"] = pts["spd"].shift(-shift)
        pts[f"azm_next_{shift}"] = pts["azm"].shift(-shift)
        pts[f"id_next_{shift}"]  = pts["randomized_id"].shift(-shift)

    seg = pts[(pts["randomized_id"] == pts["id_next_1"])].copy()
    seg["s_m"] = [haversine_m(a,b,c,d) for a,b,c,d in zip(seg["lat"],seg["lng"],seg["lat_next_1"],seg["lng_next_1"])]
    seg["bearing_pair"] = [bearing_deg(a,b,c,d) for a,b,c,d in zip(seg["lat"],seg["lng"],seg["lat_next_1"],seg["lng_next_1"])]
    seg["bearing_dev_pair"] = angle_diff_deg(seg["azm"], seg["bearing_pair"])
    # продольная перегрузка (без времени): (v2^2 - v1^2) / (2*s)
    seg["a_long"] = (seg["spd_next_1"]**2 - seg["spd"]**2) / (2.0 * seg["s_m"].replace(0, np.nan))
    # тройки для поперечной (упрощение: радиус через chord/angle)
    # Оставим как future-work; для HCT используем кривизну ребра, если mm доступен.
else:
    seg = None

print("Segments (approx) shape:", None if seg is None else seg.shape)

In [ ]:
# 4) Edge-level агрегаты (если есть match)
edge_df = None
if mm is not None:
    edge_cols = ["u","v","key","highway","oneway","length"]
    agg = pt.drop(columns=edge_cols, errors="ignore").join(mm[edge_cols], how="left")
    # Метрики по скорости и стопам
    g = agg.groupby(["u","v","key","highway","oneway"], dropna=False)
    edge_df = g.agg(
        n_obs=("spd_clip","size"),
        n_ids=("randomized_id", "nunique"),
        p50_spd=("spd_clip","median"),
        p85_spd=("spd_clip", lambda s: s.quantile(0.85)),
        p95_spd=("spd_clip", lambda s: s.quantile(0.95)),
        stop_rate=("stop_flag","mean"),
        bearing_dev_mean=("bearing_dev","mean"),
        dist2node_med=("dist2node","median")
    ).reset_index()
    edge_df["freeflow"] = edge_df["p85_spd"]
    edge_df["congestion"] = 1.0 - (edge_df["p50_spd"] / edge_df["freeflow"].replace(0,np.nan))
    edge_df.to_parquet(EDGE_FEATURES_PARQUET, index=False)
    print("Saved:", EDGE_FEATURES_PARQUET)
else:
    print("Edge-level features skipped (no map-matching).")

In [ ]:
# 5) Hex-level агрегаты
hex_df = None
if pt["h3"].notna().any():
    g = pt.groupby("h3")
    hex_df = g.agg(
        n_obs=("spd_clip","size"),
        n_ids=("randomized_id","nunique"),
        p50_spd=("spd_clip","median"),
        p85_spd=("spd_clip", lambda s: s.quantile(0.85)),
        stop_rate=("stop_flag","mean"),
        bearing_dev_mean=("bearing_dev","mean")
    ).reset_index()
    hex_df["freeflow"] = hex_df["p85_spd"]
    hex_df["congestion"] = 1.0 - (hex_df["p50_spd"] / hex_df["freeflow"].replace(0,np.nan))
    hex_df.to_parquet(HEX_FEATURES_PARQUET, index=False)
    print("Saved:", HEX_FEATURES_PARQUET)
else:
    print("Hex-level features skipped (no h3).")